In [6]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import warnings
import seaborn as sns

warnings.filterwarnings('ignore')
%matplotlib inline
#device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 모의 1: GAS MEAN TEMP 만 다루기 위한 데이터 전처리

### 예시 : 1월 데이터

In [7]:
# df = pd.read_csv('C:/Users/Sejong/Desktop/한화오션/모의csv파일/general_pivot_1.csv', encoding = 'cp949')
# df = df.drop([df.columns[i] for i in range(3,38,2)], axis = 1)
# df['kst'] = [df['kst'][i][11:] for i in range(0,88194)]
# df.rename(columns = {'癤풼s_date' : 'date'}, inplace = True)
# df_gas = df.iloc[:, :8] # gas temerature data frame
# df_gas_mean = pd.concat([df.iloc[:, :2], df.iloc[:,7]], axis = 1)
# df_gas_mean_new = {'date' : [],
#                    'kst' : [],
#                    'GAS MEAN TEMP' :[]}

# for i in range(0,len(df_gas_mean['date']),2): # 0, 2, 4,...
#     MEAN = (df_gas_mean['ME(S) CYL EXH GAS MEAN TEMP'][i] + df_gas_mean['ME(S) CYL EXH GAS MEAN TEMP'][i+1]) / 2
#     df_gas_mean_new['date'].append(df_gas_mean['date'][i])
#     df_gas_mean_new['kst'].append(df_gas_mean['kst'][i])
#     df_gas_mean_new['GAS MEAN TEMP'].append(MEAN)

# df_gas_mean_new = pd.DataFrame(df_gas_mean_new)
# df_gas_mean_new

### 1-8월 까지의 데이터 GAS MEAN TEMP 만 뽑아서 dataframe 작성

In [8]:
for idx in range(1,9): # 1월부터 8월까지
    df = pd.read_csv('./모의csv파일/general_pivot_{}.csv'.format(idx), encoding = 'cp949')
    df = df.drop([df.columns[i] for i in range(3,38,2)], axis = 1)
    df['kst'] = [df['kst'][i][11:] for i in range(0,len(df['kst']))]
    df.rename(columns = {'癤풼s_date' : 'date'}, inplace = True)
    df_gas = df.iloc[:, :8] # gas temerature data frame
    df_gas_mean = pd.concat([df.iloc[:, :2], df.iloc[:,7]], axis = 1)
    globals()['df_gas_mean_new_' + str(idx)] = {'date' : [],
                    'kst' : [],
                    'GAS MEAN TEMP' :[]}
    
    # 새로운 DataFrame 만들기
    for i in range(0,len(df_gas_mean['date']),2): # 0, 2, 4,...

        # 똑같은 시간(09:00, 09:00) 시간의 GAS TEMP 의 평균을 구해서 하나로 합침
        MEAN = (df_gas_mean['ME(S) CYL EXH GAS MEAN TEMP'][i] + df_gas_mean['ME(S) CYL EXH GAS MEAN TEMP'][i+1]) / 2

        # df_gas_mean_new_{}인 DataFrame에 값 할당
        globals()['df_gas_mean_new_' + str(idx)]['date'].append(df_gas_mean['date'][i])
        globals()['df_gas_mean_new_' + str(idx)]['kst'].append(df_gas_mean['kst'][i])
        globals()['df_gas_mean_new_' + str(idx)]['GAS MEAN TEMP'].append(MEAN)
    
    globals()['df_gas_mean_new_' + str(idx)] = pd.DataFrame( globals()['df_gas_mean_new_' + str(idx)]) # df_gas_mewn_new_1,2...8

# 결측치 평균값으로 채우기
df_gas_mean_new_7['GAS MEAN TEMP'][13884:13886] = 293.81375
df_gas_mean_new_7['GAS MEAN TEMP'][13893] = 295.035
df_gas_mean_new_7['GAS MEAN TEMP'][13898:13901] = 295.217
df_gas_mean_new_7['GAS MEAN TEMP'][14916] = 270.82
df_gas_mean_new_7['GAS MEAN TEMP'][14945] = 270.72

### 일별로 데이터가 몇개 있는지 Count

#### 00시 ~ 24시 : 24 * 60 = 1440 
#### 09시 ~ 24시 : 15 * 60 = 900

In [9]:
df_gas_count_dict = { 1: [],
                2: [],
                3: [],
                4: [],
                5 : [],
                6 : [],
                7 : [],
                8 : []}
for idx in range(1,9):
    df_gas_count = globals()['df_gas_mean_new_' + str(idx)].loc[:,'date'].value_counts(sort = False)
    for j in range(1,len(df_gas_count)+1):
        df_gas_count_dict[idx].append(df_gas_count.iloc[j-1])

print(df_gas_count_dict)

{1: [900, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1439, 1440, 1440, 1440, 1440, 1438, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440], 2: [1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440], 3: [1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1439, 1439, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440], 4: [1440, 1440, 1440, 1440, 1440, 1440, 1438, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440], 5: [1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1438, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440], 6: [1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1440, 1438,

#### 1월 Data EDA

In [10]:
df_gas_mean_new_1.describe()

,date,GAS MEAN TEMP
count,4.409700e+04,44097.000000
mean,2.023012e+07,262.060428
std,8.842902e+00,61.242246
min,2.023010e+07,49.405500
25%,2.023011e+07,276.027000
50%,2.023012e+07,280.048000
75%,2.023012e+07,283.949500
max,2.023013e+07,313.038000


#### window size : 1시간 으로 slicing -> 간격 : 60, overlap : 10

In [ ]:
mean_temp = []
#hyper parameter
DATA = df_gas_mean_new_7
LEN = len(DATA['GAS MEAN TEMP'])

# 몫과 나머지 구하기
share, remainder = (LEN // 60), (LEN % 60)
print("share, remainder : ", share, remainder)

for i in range(share):
    mean = round(DATA['GAS MEAN TEMP'][(i)*60:(i+1)*60].mean(),2) # (0,60),(60,120)
    mean_temp.append(mean)

if remainder != 0:
    mean_temp.append(DATA['GAS MEAN TEMP'][share * 60:].mean())

print("mean_temp length : ", len(mean_temp))

plt.figure(figsize = (30,10))
x = np.arange(1,len(mean_temp)+1)
sns.lineplot(x = x, y = mean_temp)
plt.show()

#### 2월부터 7월까지 1시간 단위로 자른 평균 데이터

In [108]:
MEAN_TEMP_2_to_7 = []
for i in range(2,8):
    dataset = globals()['df_gas_mean_new_{}'.format(i)]
    LEN = len(dataset['GAS MEAN TEMP'])
    share, remainder = LEN // 60, LEN % 60

    for i in range(share):
        MEAN_TEMP_2_to_7.append(round(dataset['GAS MEAN TEMP'][(i)*60:(i+1)*60].mean(), 2))

    if remainder != 0:
        MEAN_TEMP_2_to_7.append(round(dataset['GAS MEAN TEMP'][share * 60:].mean(),2))

print(np.mean(MEAN_TEMP_2_to_7), np.std(MEAN_TEMP_2_to_7))


239.72498595505618 82.41515430848148


In [128]:
L = np.array([1.0,2.0,3.0,4.0])
print(L + 42)

[43. 44. 45. 46.]


In [126]:
def linear_function(data):
    return data * 0.071 + 260
print(linear_function(L))

[260.071 260.142 260.213 260.284]


#### 모의를 위한 random 데이터 추출

In [166]:
random_temp = np.random.choice(MEAN_TEMP_2_to_7, 720)
random_temp = random_temp + 52
random_temp = linear_function(random_temp)
random_temp.mean()

280.71223833333335

In [167]:
random_temp.shape

(720,)

In [169]:
x = np.arange(1,721)
df = pd.DataFrame({
    'index' : x,
    'temp' : np.array(random_temp)
})
df

,index,temp
0,1,284.72504
1,2,282.75834
2,3,284.37288
3,4,272.55067
4,5,284.10237
...,...,...
715,716,283.29226
716,717,267.35773
717,718,281.97947
718,719,284.33454


#### 통계 분석 (기울기, y 절편)

In [171]:
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols

In [176]:
model = ols('temp ~ index', data = df).fit() 

In [177]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   temp   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     7.198
Date:                Thu, 21 Sep 2023   Prob (F-statistic):            0.00747
Time:                        15:11:11   Log-Likelihood:                -2294.8
No. Observations:                 720   AIC:                             4594.
Df Residuals:                     718   BIC:                             4603.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    279.6945      0.438    638.752      0.000     278.835     280.554
index          0.0028      0.001      2.683      0.007       0.001       0.005
==============================================================================
Omnibus:                      168.127   Durbin-Watson:                   2.095
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              296.589
Skew:                          -1.537   Prob(JB):                     3.95e-65
Kurtosis:                       3.661   Cond. No.                         833.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
plt.figure(figsize = (300,50))
x = np.arange(1,len(random_temp)+1)
sns.regplot(x = x, y = random_temp)
plt.show()

#### 본 데이터와 random 데이터 분포 비교

In [ ]:
print(np.mean(random_temp), np.std(random_temp))
plt.figure(figsize = (300,50))
x = np.arange(1,len(random_temp)+1)
sns.histplot(random_temp)
plt.show()

In [ ]:
plt.figure(figsize = (300,50))
sns.histplot(MEAN_TEMP_2_to_7)
plt.show()

In [13]:
# for i in range(8):
#     pivot_data = pd.read_csv('./csv파일/2451_general_pivot_{}.csv'.format(i+1), encoding = 'cp949')
#     pkg_data = pd.read_csv('./csv파일/2451_general_pkg_list_{}.CSV'.format(i+1), encoding = 'cp949')
#     pivot_df = pd.DataFrame(pivot_data)
#     pkg_df = pd.DataFrame(pkg_data) 
#     one = pivot_df.loc[:, 'ds_date'].value_counts(sort=False)
#     two = pkg_df.loc[:, 'ds_date'].value_counts(sort=False)
#     result = pd.concat([one, two], axis =1 )
#     print(result)

In [14]:
# for i in range(8):
#     pivot_data = pd.read_csv('./csv파일/2451_general_pivot_{}.csv'.format(i+1), encoding = 'cp949')
    
#     pivot_df = pd.DataFrame(pivot_data)

#     one = pivot_df.loc[:, 'ds_date'].value_counts(sort=False)

#     print(one)

In [15]:
# for i in range(8):
#     pivot_data = pd.read_csv('./csv파일/2451_general_pkg_list_{}.csv'.format(i+1), encoding = 'cp949')
#     pkg_data = pd.read_csv('./한화오션_예지보전_데이터/2451_general_pkg_list_{}.csv'.format(i+1), encoding = 'cp949')
#     pivot_df = pd.DataFrame(pivot_data)
#     pkg_df = pd.DataFrame(pkg_data)
#     one = pivot_df.loc[:, 'ds_date'].value_counts(sort=False)
#     two = pkg_df.loc[:, 'ds_date'].value_counts(sort=False)
#     result = pd.concat([one, two], axis =1 )
#     print(result)